In [11]:
import numpy as np
import pandas as pd
import requests
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

In [12]:
def fetch_binance_data(symbol, interval='1h', limit=1000):
    url = "https://api.binance.com/api/v3/klines"
    params = {'symbol': symbol, 'interval': interval, 'limit': limit}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['volume'] = df['volume'].astype(float)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    return df

In [13]:
def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def create_features(df):
    df['SMA_50'] = df['close'].rolling(window=50).mean()
    df['SMA_200'] = df['close'].rolling(window=200).mean()
    df['RSI'] = compute_rsi(df['close'], 14)
    df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['Bollinger_High'] = df['close'].rolling(window=20).mean() + 2 * df['close'].rolling(window=20).std()
    df['Bollinger_Low'] = df['close'].rolling(window=20).mean() - 2 * df['close'].rolling(window=20).std()
    df['volume_change'] = df['volume'].pct_change()
    
    # Lag features
    for i in range(1, 11):
        df[f'close_lag_{i}'] = df['close'].shift(i)
    
    df.dropna(inplace=True)  # Remove rows with NaN values

In [14]:
def create_feature_label_data(df):
    df['future_close'] = df['close'].shift(-7)  # Predict price 7 days ahead
    df.dropna(inplace=True)  # Remove rows with NaN values

    features = df[['SMA_50', 'SMA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'MACD_signal', 'Bollinger_High', 'Bollinger_Low', 'volume_change'] + [f'close_lag_{i}' for i in range(1, 11)]]
    labels = df['future_close']
    return features, labels

def build_and_train_model(features, labels):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    
    X_train, X_test, y_train, y_test = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Mean Squared Error: {mse:.2f}")
    print(f"Model Accuracy (R² score): {r2:.2%}")
    
    return model, scaler

In [15]:
def predict_future_prices(model, scaler, df, conversion_rate, symbol):
    today = df['close_time'].max()
    
    # Set next_midnight to midnight of the next day
    next_midnight = (today + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
    
    future_dates = pd.date_range(start=next_midnight, periods=168, freq='h')  # Updated freq to 'h'
    future_df = pd.DataFrame(index=future_dates)
    
    # Ensure features_count matches
    features_count = len(df[['SMA_50', 'SMA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'MACD_signal', 'Bollinger_High', 'Bollinger_Low', 'volume_change'] + [f'close_lag_{i}' for i in range(1, 11)]].columns)
    
    last_features = df[['SMA_50', 'SMA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'MACD_signal', 'Bollinger_High', 'Bollinger_Low', 'volume_change'] + [f'close_lag_{i}' for i in range(1, 11)]].iloc[-1].values
    last_features = np.reshape(last_features, (1, features_count))  # Reshape for prediction
    
    future_predictions = []
    
    for _ in range(168):  # Predict for the next 168 hours
        # Predict the price for the next hour
        scaled_last_features = scaler.transform(pd.DataFrame(last_features, columns=df[['SMA_50', 'SMA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'MACD_signal', 'Bollinger_High', 'Bollinger_Low', 'volume_change'] + [f'close_lag_{i}' for i in range(1, 11)]].columns))
        prediction = model.predict(scaled_last_features)
        future_predictions.append(prediction[0])
        
        # Update last_features for next prediction
        last_features = np.roll(last_features, shift=-1, axis=1)  # Roll features left
        last_features[0, -1] = prediction[0]  # Update last feature with the new prediction

    future_df['predicted_close'] = future_predictions
    future_df['predicted_close_inr'] = future_df['predicted_close'] * conversion_rate

    # Find best times to buy and sell
    min_index = future_df['predicted_close'].idxmin()
    max_index = future_df['predicted_close'].idxmax()

    best_time_to_buy = future_df.loc[min_index].name
    best_time_to_sell = future_df.loc[max_index].name
    best_price_to_buy = future_df.loc[min_index, 'predicted_close']
    best_price_to_sell = future_df.loc[max_index, 'predicted_close']
    best_price_to_buy_inr = best_price_to_buy * conversion_rate
    best_price_to_sell_inr = best_price_to_sell * conversion_rate

    # Format the datetime to 'YEAR-MONTH-DATE  HOUR:MINUTE PM/AM'
    best_time_to_buy = best_time_to_buy.strftime('%Y-%m-%d %I:%M %p')
    best_time_to_sell = best_time_to_sell.strftime('%Y-%m-%d %I:%M %p')

    future_df['formatted_date'] = future_df.index.strftime('%Y-%m-%d %I:%M %p')

    print(f"\nBest time to buy {symbol}: {best_time_to_buy} at price ${best_price_to_buy:.8f} ({best_price_to_buy_inr:.2f} INR)")
    print(f"Best time to sell {symbol}: {best_time_to_sell} at price ${best_price_to_sell:.8f} ({best_price_to_sell_inr:.2f} INR)")

    return future_df

In [16]:
def get_conversion_rate():
    url = "https://api.exchangerate-api.com/v4/latest/USD"
    response = requests.get(url)
    data = response.json()
    conversion_rate = data['rates'].get('INR', 1)  # Default to 1 if INR rate is not found
    return conversion_rate

In [17]:
def retrain_model(symbol):
    # Fetch the latest data
    df = fetch_binance_data(symbol)
    
    # Create features and labels
    create_features(df)
    features, labels = create_feature_label_data(df)
    
    # Build and train the model
    model, scaler = build_and_train_model(features, labels)
    
    return model, scaler, df

In [27]:
def main():
    symbol = input("Enter the cryptocurrency symbol (e.g., BTC): ")
    symbol = symbol + 'USDT'
    model, scaler, df = retrain_model(symbol)

    conversion_rate = get_conversion_rate()
    future_df = predict_future_prices(model, scaler, df, conversion_rate, symbol)

    # Print the predicted prices in a tabular format
    print("\nPredicted Prices for {} in the next 168 hours:".format(symbol))

    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 1000)
    
    print(future_df[['formatted_date', 'predicted_close', 'predicted_close_inr']].to_string(index=False, header=True, float_format=lambda x: '{:.8f}'.format(x)))

    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')

In [28]:
if __name__ == "__main__":
    main()

Model Mean Squared Error: 141085.94
Model Accuracy (R² score): 99.10%

Best time to buy BTCUSDT: 2024-08-01 04:00 AM at price $65712.58050000 (5503428.62 INR)
Best time to sell BTCUSDT: 2024-08-06 06:00 AM at price $68700.48290000 (5753665.44 INR)

Predicted Prices for BTCUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-08-01 12:00 AM   66381.35710000     5559438.65712499
2024-08-01 01:00 AM   66158.77110000     5540797.07962500
2024-08-01 02:00 AM   66389.55850000     5560125.52437500
2024-08-01 03:00 AM   66767.02700000     5591738.51124999
2024-08-01 04:00 AM   65712.58050000     5503428.61687500
2024-08-01 05:00 AM   66139.46520000     5539180.21050000
2024-08-01 06:00 AM   66584.79900000     5576476.91625000
2024-08-01 07:00 AM   66909.21130000     5603646.44637499
2024-08-01 08:00 AM   66436.76620000     5564079.16925000
2024-08-01 09:00 AM   66648.22460000     5581788.81025000
2024-08-01 10:00 AM   67141.22550000     5623077.63562499
202

In [10]:
if __name__ == "__main__":
    main()

Model Mean Squared Error: 683.88
Model Accuracy (R² score): 97.56%

Best time to buy ETHUSDT: 2024-07-31 12:00 AM at price $3290.26700000 (275625.67 INR)
Best time to sell ETHUSDT: 2024-08-04 09:00 PM at price $3426.03610000 (286999.04 INR)

Predicted Prices for ETHUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-07-31 12:00 AM    3290.26700000      275625.66659000
2024-07-31 01:00 AM    3378.10120000      282983.53752400
2024-07-31 02:00 AM    3398.20810000      284667.89253700
2024-07-31 03:00 AM    3335.81900000      279441.55763000
2024-07-31 04:00 AM    3382.96640000      283391.09532800
2024-07-31 05:00 AM    3395.86080000      284471.25921600
2024-07-31 06:00 AM    3408.55930000      285535.01256100
2024-07-31 07:00 AM    3377.60170000      282941.69440900
2024-07-31 08:00 AM    3375.47490000      282763.53237300
2024-07-31 09:00 AM    3399.60830000      284785.18729100
2024-07-31 10:00 AM    3385.15760000      283574.65215200
2024-07-31

In [9]:
if __name__ == '__main__':
    main()

Model Mean Squared Error: 141075.50
Model Accuracy (R² score): 99.10%

Best time to buy BTCUSDT: 2024-08-01 04:00 AM at price $65726.85100000 (5504623.77 INR)
Best time to sell BTCUSDT: 2024-08-02 06:00 PM at price $68829.66100000 (5764484.11 INR)

Predicted Prices for BTCUSDT in the next 168 hours:
     formatted_date  predicted_close  predicted_close_inr
2024-08-01 12:00 AM   66378.67710000     5559214.20712499
2024-08-01 01:00 AM   66160.64260000     5540953.81775000
2024-08-01 02:00 AM   66389.55850000     5560125.52437500
2024-08-01 03:00 AM   66796.50700000     5594207.46124999
2024-08-01 04:00 AM   65726.85100000     5504623.77125000
2024-08-01 05:00 AM   66139.46520000     5539180.21050000
2024-08-01 06:00 AM   66584.79900000     5576476.91625000
2024-08-01 07:00 AM   66923.11080000     5604810.52949999
2024-08-01 08:00 AM   66438.72910000     5564243.56212500
2024-08-01 09:00 AM   66652.39460000     5582138.04775000
2024-08-01 10:00 AM   67139.96450000     5622972.02687499
202